In [1]:
import pickle
import pandas as pd
import numpy as np
import optuna

#     optuna_ex1_hyperparameters_per_dataset.py
#  -> analysis_ex1_hyperparameters.ipynb
#  -> benchmark_ex1_best_hyperparameters.py
#  -> analysis_ex1_hyperparameters_best.ipynb

In [2]:
storage = "sqlite:///optuna_databases/optuna_ex1_parameter_tuning_FG_e3_d2.sqlite3" #CHANGE


In [3]:
studies = optuna.study.get_all_study_summaries(storage=storage)

In [4]:
loaded_study = optuna.load_study(study_name=studies[0].study_name, storage=storage)
df = loaded_study.trials_dataframe()
for key, value in loaded_study.system_attrs.items():
    df[key] = value
for study in studies[1:]:
    loaded_study = optuna.load_study(study_name=study.study_name , storage=storage)
    df_new = loaded_study.trials_dataframe()
    for key, value in loaded_study.system_attrs.items():
        df_new[key] = value
    df = pd.concat([df, df_new])
df.reset_index(inplace=True, drop=True)

In [5]:
df.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [6]:
df.head(1)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,0,0.737129,2021-08-01 09:57:27.287026,2021-08-01 09:57:36.638517,0 days 00:00:09.351491,58.713307,0.98689,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.initialization_method.value_counts()

KDTmeans_init    1200
BGM_init         1200
str              1200
Kmeans_init      1200
Name: initialization_method, dtype: int64

In [8]:
np.unique(df.initialization_method)

array(['BGM_init', 'KDTmeans_init', 'Kmeans_init', 'str'], dtype=object)

In [9]:
np.unique(df["Data X"])

array(['abalone_input.pd', 'adult_input.pd', 'bank_input.pd',
       'banknote_input.pd', 'breast_cancer_input.np', 'cars_input.pd',
       'contraceptive_input.pd', 'generated6_input.np', 'hrss_input.pd',
       'iris_input.pd', 'steel_input.pd', 'students_input.pd'],
      dtype=object)

In [10]:
top = 10
df_best = pd.DataFrame()
for dataset in np.unique(df["Data X"]):
    df_new = df[df["Data X"] == dataset].sort_values(by=["user_attrs_accuracy_val"], ascending=False).head(top)
    df_best = pd.concat([df_best, df_new])
df_best.reset_index(inplace=True, drop=True)


In [11]:
df_best.shape

(120, 27)

In [12]:
df_best.head(2)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,99,0.744551,2021-08-01 10:12:49.223958,2021-08-01 10:12:58.311047,0 days 00:00:09.087089,105.335940,0.979024,2,3,least_squares_linear_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,80,0.743597,2021-08-01 11:01:19.841504,2021-08-01 11:01:29.775526,0 days 00:00:09.934022,26.374218,0.996630,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,BGM_init,NaN,NaN,NaN,0.690984,0.993941,dirichlet_process,0.0


In [13]:
pickle.dump(df_best, open("dataframes/ex1_df_top10_hyperparameters_per_dataset_FG_e3_d2.pd", "wb"))
#pickle.dump(df_best, open("dataframes/ex1_df_top10_hyperparameters_per_dataset_2D_e3_d2.pd", "wb")) #CHANGE

In [14]:
df.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,49.5,0.721247,96.811629,0.990977,2.0,3.0,False,0.741603,0.721247,0.009394,0.014063,2.373486,0.214790,0.216893,0.660676,0.757818,0.0
adult_input.pd,49.5,0.820058,94.519625,0.988661,2.0,3.0,False,0.833839,0.820058,0.002728,0.015160,2.741668,0.128315,0.252740,0.368580,0.470691,0.0
bank_input.pd,49.5,0.893576,72.718241,0.988409,2.0,3.0,False,0.908610,0.893576,0.001391,0.017817,1.543526,0.347298,0.264870,0.561818,0.577128,0.0
banknote_input.pd,49.5,0.990355,99.792260,0.991196,2.0,3.0,False,0.994928,0.990355,0.001892,0.004966,2.000330,0.266298,0.212740,0.507898,0.334241,0.0
breast_cancer_input.np,49.5,0.921008,89.011376,0.985726,2.0,3.0,False,0.977689,0.921008,0.005059,0.019276,2.030817,0.257354,0.266819,0.347714,0.373375,0.0
cars_input.pd,49.5,0.837642,71.816171,0.988026,2.0,3.0,False,0.849168,0.837642,0.011844,0.018472,2.207345,0.295656,0.299062,0.716310,0.557731,0.0
contraceptive_input.pd,49.5,0.476611,83.289465,0.989048,2.0,3.0,False,0.516544,0.476611,0.021064,0.023459,2.124954,0.186198,0.243814,0.443408,0.737701,0.0
generated6_input.np,49.5,0.934682,94.293090,0.987885,2.0,3.0,False,0.939774,0.934682,0.011175,0.013023,3.212625,0.245182,0.192469,0.620360,0.509176,0.0
hrss_input.pd,49.5,0.772569,96.983612,0.989518,2.0,3.0,False,0.774025,0.772569,0.003079,0.004864,2.373430,0.271384,0.307401,0.633489,0.573778,0.0


In [15]:
df_best.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,45.2,0.741106,79.569444,0.983270,2.0,3.0,False,0.755853,0.741106,0.004167,0.010109,NaN,NaN,NaN,0.690984,0.993941,0.0
adult_input.pd,57.8,0.839109,102.747275,0.981776,2.0,3.0,False,0.840047,0.839109,0.002328,0.003966,3.054080,0.073167,0.356302,NaN,NaN,NaN
bank_input.pd,71.0,0.907240,60.514409,0.990251,2.0,3.0,False,0.910132,0.907240,0.001554,0.002468,0.767410,0.448480,0.261945,0.374489,0.458089,0.0
banknote_input.pd,52.2,0.999344,128.963570,0.992441,2.0,3.0,False,0.999490,0.999344,0.000634,0.001136,1.840128,0.368680,0.174474,0.312761,0.243509,0.0
breast_cancer_input.np,59.4,0.942378,109.108423,0.986425,2.0,3.0,False,0.982836,0.942378,0.004622,0.018857,1.222096,0.283576,0.121287,0.350092,0.214208,0.0
cars_input.pd,66.5,0.908275,111.947279,0.983131,2.0,3.0,False,0.920390,0.908275,0.007144,0.011395,NaN,NaN,NaN,0.845475,0.926563,0.0
contraceptive_input.pd,54.9,0.535443,72.893356,0.985582,2.0,3.0,False,0.591242,0.535443,0.012085,0.019310,NaN,NaN,NaN,NaN,NaN,NaN
generated6_input.np,65.1,0.968600,108.171731,0.986677,2.0,3.0,False,0.971353,0.968600,0.004434,0.005465,3.563866,0.215078,0.193654,0.343637,0.469480,0.0
hrss_input.pd,56.7,0.778016,130.115336,0.987273,2.0,3.0,False,0.778854,0.778016,0.003866,0.004184,NaN,NaN,NaN,0.670108,0.757414,0.0


In [16]:
df["params_use_2_dim_gate_based_on"].value_counts(dropna=False)

NaN    4800
Name: params_use_2_dim_gate_based_on, dtype: int64

In [17]:
df_best["params_optimization_method"].value_counts()

ridge_regression                   59
least_squares_linear_regression    48
lasso_regression                   13
Name: params_optimization_method, dtype: int64

In [18]:
df_best.groupby(["params_optimization_method"]).median()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
params_optimization_method,,,,,,,,,,,,,,,,,
lasso_regression,62.0,0.839334,92.179746,0.985302,2.0,3.0,False,0.839931,0.839334,0.001610,0.003954,0.824356,0.426253,0.129430,NaN,NaN,NaN
least_squares_linear_regression,57.5,0.942006,92.374768,0.986529,2.0,3.0,False,0.970000,0.942006,0.004223,0.010240,3.297155,0.184299,0.248566,0.548914,0.571630,0.0
ridge_regression,64.0,0.839467,98.906816,0.986770,2.0,3.0,False,0.842152,0.839467,0.005152,0.010655,3.053379,0.157491,0.281216,0.420200,0.571565,0.0


In [20]:
df_best.groupby(["params_optimization_method"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
params_optimization_method,,,,,,,,,,,,,,,,,
lasso_regression,63.846154,0.748208,86.129834,0.984515,2.0,3.0,False,0.756270,0.748208,0.004511,0.009171,0.824356,0.426253,0.129430,NaN,NaN,NaN
least_squares_linear_regression,51.750000,0.861607,94.054996,0.987352,2.0,3.0,False,0.878175,0.861607,0.004964,0.012220,3.172670,0.195441,0.267134,0.525082,0.541967,0.0
ridge_regression,60.694915,0.788677,92.608457,0.986571,2.0,3.0,False,0.812043,0.788677,0.007222,0.012849,2.690768,0.215031,0.264677,0.477155,0.583699,0.0


In [19]:
df_best["params_use_2_dim_gate_based_on"].value_counts()

Series([], Name: params_use_2_dim_gate_based_on, dtype: int64)